# Task overview/Game Rules

*The game consists of a board of cells that are either on or off. One creates an initial configuration of these on/off states and observes how it evolves. There are four simple rules to determine the next state of the game board, given the current state:*

* #### Overpopulation: if a living cell is surrounded by more than three living cells, it dies.
* #### Stasis: if a living cell is surrounded by two or three living cells, it survives.
* #### Underpopulation: if a living cell is surrounded by fewer than two living cells, it dies.
* #### Reproduction: if a dead cell is surrounded by exactly three cells, it becomes a live cell.

![](https://natureofcode.com/book/imgs/chapter07/ch07_01.png)

# Approach overview


### Iterative model
The basic idea is to train a model that can predict start state from stop state(with delta=1) and apply this model delta times.

The model itself can be any (not only neural networks).


![](https://i.ibb.co/y0fMSCg/simple-model-diagram.png)


### Iterative model with Encoder/Decoder

Using an encoder and a decoder allows more information to be conveyed to the next step (not just one channel).

![](https://i.ibb.co/MRvZbYj/model-diagram.png)


### Iterative CNN model with Encoder/Decoder

The stop and start state can be viewed as single-channel images(so we can use CV approaches)

Input - 1 channel image

Encoder in - 1 channel image | out - N channel image

ReverseOneIterationModel - inp N channel image | out - N channel image

Decoder in - N channel image | out - 1 channel image

# Model

In [ ]:
import torch
import torch.nn as nn


class OneIterationReverseNet(nn.Module):
    def __init__(self, info_ch, ch):
        super().__init__()
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv2d(info_ch, ch, 5, padding=4, padding_mode='circular')
        self.conv2 = nn.Conv2d(ch, ch, 3, )
        self.conv3 = nn.Conv2d(ch, info_ch, 3)
        
        
    def forward(self, inp):
        x = self.relu(self.conv1(inp))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        return x
      
        
class ReverseModel(nn.Module):
    def __init__(self, info_ch=64, ch=128):
        super().__init__()
        self.relu = nn.ReLU()
        self.encoder = nn.Conv2d(1, info_ch, 7, padding=3, padding_mode='circular')# you can use other model
        self.reverse_one_iter = OneIterationReverseNet(info_ch, ch)# you can use other model
        self.decoder = nn.Conv2d(info_ch, 1, 3, padding=1, padding_mode='circular')# you can use other model
        
    
    def forward(self, stop, delta):
        x = self.relu(self.encoder(stop-0.5))
        
        for i in range(delta.max().item()):
            y = self.reverse_one_iter(x)
            
            # this 2 lines allow use samples with different delta in one batch
            mask = (delta > i).reshape(-1,1,1,1)
            x = x*(~mask).float() + y*mask.float()
        
        x = self.decoder(x)
        
        return x 

# Load data

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
train_val = pd.read_csv('/kaggle/input/conways-reverse-game-of-life-2020/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/conways-reverse-game-of-life-2020/test.csv', index_col='id')

train, val = train_test_split(train_val, test_size=0.2, shuffle=True, random_state=42, stratify=train_val['delta'])

# Dataset

In [ ]:
from torch.utils.data import DataLoader, Dataset
from torch import FloatTensor, LongTensor


def line2grid_tensor(data, device='cuda'):
    grid = data.to_numpy().reshape((data.shape[0], 1, 25, 25))
    return FloatTensor(grid).to(device)


class TaskDataset(Dataset):
    def __init__(self, data, device='cuda'):
        self.delta = LongTensor(data['delta'].to_numpy()).to(device)
        if data.shape[1] == 1251: 
            self.start = line2grid_tensor(data.iloc[:,1:626], device)
            self.stop = line2grid_tensor(data.iloc[:,626:], device)
        else:
            self.start = None
            self.stop = line2grid_tensor(data.iloc[:,1:], device)
        
    def __len__(self):
        return len(self.delta)

    def __getitem__(self, idx):
        if self.start is None:
            return {'stop': self.stop[idx], 'delta': self.delta[idx]}
        return {'start': self.start[idx], 'stop': self.stop[idx], 'delta': self.delta[idx]}

In [ ]:
dataset_train = TaskDataset(train)
dataloader_train = DataLoader(dataset_train, batch_size=128, shuffle=True)

dataset_val = TaskDataset(val)
dataloader_val = DataLoader(dataset_val, batch_size=128, shuffle=False)

dataset_test = TaskDataset(test)
dataloader_test = DataLoader(dataset_test, batch_size=128, shuffle=False)

# Train Loop

In [ ]:
from catalyst.dl import SupervisedRunner
from catalyst.dl.callbacks import CriterionCallback, EarlyStoppingCallback, AccuracyCallback
from catalyst.contrib.nn.optimizers import RAdam, Lookahead

import collections

runner = SupervisedRunner(device='cuda', input_key=['stop', 'delta'], )

loaders = {'train': dataloader_train, 'valid': dataloader_val}#collections.OrderedDict({'train': dataloader_train, 'valid': dataloader_val})

model = ReverseModel()

optimizer = Lookahead(RAdam(params=model.parameters(), lr=1e-3))

criterion = {"bce": nn.BCEWithLogitsLoss()}

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.25, patience=2)

callbacks = [
        CriterionCallback(input_key='start', prefix="loss", criterion_key="bce"),
        EarlyStoppingCallback(patience=5),
    ]

runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    callbacks=callbacks,
    logdir="./logs",
    num_epochs=999,
    main_metric="loss",
    minimize_metric=True,
    verbose=True,
)

# Prediction / LB estimate

### Load best model

In [ ]:
best_model = ReverseModel().to('cuda')
best_model.load_state_dict(torch.load('logs/checkpoints/best.pth')['model_state_dict'])

### LB estimate

In [ ]:
import numpy as np

def predict_batch(model, batch):
    model.eval()
    with torch.no_grad():
        prediction = model(batch['stop'], batch['delta'])
        prediction = torch.sigmoid(prediction).detach().cpu().numpy()
        return prediction

    
def predict_loader(model, loader):
    predict = [predict_batch(model, batch) for batch in loader]
    predict = np.concatenate(predict)
    return predict


def validate_loader(model, loader, lb_delta=None, threshold=0.5):
    prediction_val = predict_loader(best_model, loader)
    y_val = loader.dataset.start.detach().cpu().numpy()
    delta_val = loader.dataset.delta.detach().cpu().numpy()

    score = ((prediction_val > threshold) == y_val).mean(axis=(1,2,3))
    print(f'All data accuracy: {score.mean()}')
        
    delta_socre = {}
    for i in range(1, 6):
        delta_socre[i] = score[delta_val==i].mean()#print(f'delta={i} accuracy: {score[delta_val==i].mean()}')
        print(f'delta={i} accuracy: {delta_socre[i]}')
        
    if lb_delta is not None:
        lb_delta = lb_delta.value_counts(normalize=True)
        test_score = sum([lb_delta[i]*delta_socre[i] for i in range(1,6)])
        print(f'VAL score         : {1-score.mean()}')
        print(f'LB  score estimate: {1-test_score}')
    
    
def make_submission(prediction, threshold=0.5, sample_submission_path='/kaggle/input/conways-reverse-game-of-life-2020/sample_submission.csv'):
    prediction = (prediction > threshold).astype(int).reshape(-1, 625)
    
    sample_submission = pd.read_csv(sample_submission_path, index_col='id')
    sample_submission.iloc[:] = prediction
    return sample_submission

In [ ]:
validate_loader(best_model, dataloader_val, test['delta'])

### submission

In [ ]:
prediction_test = predict_loader(best_model, dataloader_test)
submission = make_submission(prediction_test)
submission.to_csv('submission.csv')
submission